# CONVERT TOPOLOGIC GRAPH INTO A NEO4J GRAPH

### This script builds a 3D model of a two-storey building as a CellComplex with apertures (doors and windows) and contents (furniture), converts it into a topologic graph, and, lastly, converts the topologic graph into a NEO4J graph. This is an experimental feature and we welcome your feedback on it.

#### This script requires topologicpy version 0.7.56 or newer

## Import Libraries

In [1]:
# This is only needed on the software development machine. If you are a user, you can skip this step.
import sys
sys.path.append("C:/Users/sarwj/OneDrive - Cardiff University/Documents/GitHub/topologicpy/src")

In [2]:
# Make sure you have pip installed neo4j
from topologicpy.Vertex import Vertex
from topologicpy.Edge import Edge
from topologicpy.Wire import Wire
from topologicpy.Face import Face
from topologicpy.Shell import Shell
from topologicpy.Cell import Cell
from topologicpy.CellComplex import CellComplex
from topologicpy.Cluster import Cluster
from topologicpy.Topology import Topology
from topologicpy.Aperture import Aperture
from topologicpy.Graph import Graph
from topologicpy.Dictionary import Dictionary
from topologicpy.Helper import Helper
from topologicpy.Plotly import Plotly
from topologicpy.Neo4j import Neo4j
from getpass import getpass
import uuid
import hashlib
import json

## Check the topologicpy version

In [3]:
print("This script requires topologicpy version 0.7.57 or newer.")
print("You are using topologicpy version:",Helper.Version())

This script requires topologicpy version 0.7.57 or newer.
You are using topologicpy version: 0.7.57


In [4]:
def assignTypesAndLabels(cellComplex, dictionary, typeKey="type", labelKey="label", nameKey="name", volumeKey="hasVolume", areaKey="hasArea", orientationKey="hasOrientation"):
    from topologicpy.Topology import Topology
    from topologicpy.Vector import Vector
    decomp = CellComplex.Decompose(cellComplex)
    keys = decomp.keys()
    for key in keys:
        elements = decomp[key]
        n = max(len(str(len(elements))),4)
        for i, element in enumerate(elements):
            d = Topology.Dictionary(element)
            labelValue = Dictionary.ValueAtKey(d, labelKey)
            typeValue = Dictionary.ValueAtKey(d, typeKey)
            nameValue = Dictionary.ValueAtKey(d, nameKey)
            uuid = Topology.UUID(element, namespace="http://github.com/wassimj/topologicpy/resources")
            if not typeValue:
                typeValue = dictionary[key]
            if labelValue == None:
                labelValue = str(uuid)
            print("Key.Lower:", key.lower())
            if "cell" in key.lower():
                volumeValue = Cell.Volume(element)
                d = Dictionary.SetValueAtKey(d, typeKey, typeValue)
                d = Dictionary.SetValueAtKey(d, volumeKey, volumeValue)
                d = Dictionary.SetValueAtKey(d, labelKey, labelValue)
                contents = Topology.Contents(element)
                m = max(len(str(len(contents))),4)
                for content in contents:
                    d1 = Topology.Dictionary(content)
                    labelValue1 = Dictionary.ValueAtKey(d1, labelKey)
                    if labelValue1 == None:
                        labelValue1 = str(uuid)
                    d1 = Dictionary.SetValueAtKey(d1, labelKey, labelValue1)
                    d1 = Dictionary.SetValueAtKey(d1, typeKey, "Content")
                    content = Topology.SetDictionary(content, d1)
            else:
                if "face" in key.lower() or "aperture" in key.lower():
                    areaValue = Face.Area(element)
                if "vertical" in key.lower():
                    fn = Face.Normal(element)
                    orientationValue = Vector.CompassAngle(fn, [0,1,0], mantissa=6, tolerance=0.0001)
                if "externalverticalface" in key.lower():
                    nameValue = "External Wall"
                elif "internalverticalface" in key.lower():
                    nameValue = "Internal Wall"
                elif "internalhorizontalface" in key.lower():
                    nameValue = "Internal Slab"
                elif "bottomhorizontalface" in key.lower():
                    nameValue = "Ground Slab"
                elif "tophorizontalface" in key.lower():
                    nameValue = "Roof Slab"
                if "exteriorverticalaperture" in key.lower():
                    nameValue = "External Window"
                elif "interiorverticalaperture" in key.lower():
                    nameValue = "Internal Opening"
                elif "interiorhorizontalaperture" in key.lower():
                    nameValue = "Internal Floor Opening"
                elif "bottomhorizontalaperture" in key.lower():
                    nameValue = "Ground Opening"
                elif "tophorizontalaperture" in key.lower():
                    nameValue = "Roof Skylight"
                print("Name Value:", nameValue)
                d = Dictionary.SetValueAtKey(d, typeKey, typeValue)
                d = Dictionary.SetValueAtKey(d, labelKey, labelValue)
                d = Dictionary.SetValueAtKey(d, areaKey, areaValue)
                d = Dictionary.SetValueAtKey(d, orientationKey, orientationValue)
                d = Dictionary.SetValueAtKey(d, nameKey, nameValue)
            element = Topology.SetDictionary(element, d)

## Build the House

In [5]:
selectors = []
namespace = "http://github.com/wassimj/topologicpy/resources"
lr_origin = Vertex.ByCoordinates(0,0,0)
living_room = Cell.Prism(origin=lr_origin, width=3, length=5, height=3, placement="lowerleft")
cen = Topology.Centroid(living_room)
uuid = Topology.UUID(living_room, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Living Room"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

kt_origin = Vertex.ByCoordinates(0,5,0)
kitchen = Cell.Prism(origin=kt_origin, width=3, length=4, height=3, placement="lowerleft")
cen = Topology.Centroid(kitchen)
uuid = Topology.UUID(kitchen, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Kitchen"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

cr_origin = Vertex.ByCoordinates(3,2,0)
corridor = Cell.Prism(origin=cr_origin, width=2, length=6, height=3, placement="lowerleft")
cen = Topology.Centroid(corridor)
uuid = Topology.UUID(corridor, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Corridor"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

br_origin = Vertex.ByCoordinates(5,0,0)
bedroom = Cell.Prism(origin=br_origin, width=4, length=4, height=3, placement="lowerleft")
cen = Topology.Centroid(bedroom)
uuid = Topology.UUID(bedroom, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Bedroom"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

bt_origin = Vertex.ByCoordinates(5,4,0)
bathroom = Cell.Prism(origin=bt_origin, width=3, length=2, height=3, placement="lowerleft")
cen = Topology.Centroid(bathroom)
uuid = Topology.UUID(bathroom, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Bathroom"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

ho_origin = Vertex.ByCoordinates(5,6,0)
home_office = Cell.Prism(origin=ho_origin, width=4, length=3, height=3, placement="lowerleft")
cen = Topology.Centroid(home_office)
uuid = Topology.UUID(home_office, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Home Office"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

sw_origin = Vertex.ByCoordinates(3.2,8,0)
stairwell = Cell.Prism(origin=sw_origin, width=1.6, length=3.2, height=6, wSides=2, placement="lowerleft")
cen = Topology.Centroid(stairwell)
uuid = Topology.UUID(stairwell, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Stairwell"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

gd_flr_rooms = [living_room, kitchen, corridor, bedroom, bathroom, home_office]

lr_origin = Vertex.ByCoordinates(0,0,3)
living_room = Cell.Prism(origin=lr_origin, width=3, length=5, height=3, placement="lowerleft")
cen = Topology.Centroid(living_room)
uuid = Topology.UUID(living_room, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Living Room"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

kt_origin = Vertex.ByCoordinates(0,5,3)
kitchen = Cell.Prism(origin=kt_origin, width=3, length=4, height=3, placement="lowerleft")
cen = Topology.Centroid(kitchen)
uuid = Topology.UUID(kitchen, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Kitchen"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

cr_origin = Vertex.ByCoordinates(3,2,3)
corridor = Cell.Prism(origin=cr_origin, width=2, length=6, height=3, placement="lowerleft")
cen = Topology.Centroid(corridor)
uuid = Topology.UUID(corridor, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Corridor"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

br_origin = Vertex.ByCoordinates(5,0,3)
bedroom = Cell.Prism(origin=br_origin, width=4, length=4, height=3, placement="lowerleft")
cen = Topology.Centroid(bedroom)
uuid = Topology.UUID(bedroom, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Bedroom"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

bt_origin = Vertex.ByCoordinates(5,4,3)
bathroom = Cell.Prism(origin=bt_origin, width=3, length=2, height=3, placement="lowerleft")
cen = Topology.Centroid(bathroom)
uuid = Topology.UUID(bathroom, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Bathroom"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)

ho_origin = Vertex.ByCoordinates(5,6,3)
home_office = Cell.Prism(origin=ho_origin, width=4, length=3, height=3, placement="lowerleft")
cen = Topology.Centroid(home_office)
uuid = Topology.UUID(home_office, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Home Office"])
cen = Topology.SetDictionary(cen, d)
selectors.append(cen)


fst_flr_rooms = [living_room, kitchen, corridor, bedroom, bathroom, home_office]

house = CellComplex.ByCells(gd_flr_rooms+fst_flr_rooms+[stairwell])

# Build the doors
orig = Vertex.ByCoordinates(0.2,5,0.05)
d2 = Face.Rectangle(origin=orig, width=2.6, length=2.10, placement="lowerleft")
d2 = Topology.Rotate(d2, origin=orig, axis=[1,0,0], angle=90)

orig = Vertex.ByCoordinates(3,3.4,0.05)
d3 = Face.Rectangle(origin=orig, width=2.10, length=1, placement="lowerleft")
d3 = Topology.Rotate(d3, origin=orig, axis=[0,1,0], angle=-90)

orig = Vertex.ByCoordinates(3,5.2,0.05)
d4 = Face.Rectangle(origin=orig, width=2.10, length=1, placement="lowerleft")
d4 = Topology.Rotate(d4, origin=orig, axis=[0,1,0], angle=-90)

orig = Vertex.ByCoordinates(5,2.8,0.05)
d5 = Face.Rectangle(origin=orig, width=2.10, length=1, placement="lowerleft")
d5 = Topology.Rotate(d5, origin=orig, axis=[0,1,0], angle=-90)

orig = Vertex.ByCoordinates(5,4.2,0.05)
d6 = Face.Rectangle(origin=orig, width=2.10, length=1, placement="lowerleft")
d6 = Topology.Rotate(d6, origin=orig, axis=[0,1,0], angle=-90)

orig = Vertex.ByCoordinates(5,6.2,0.05)
d7 = Face.Rectangle(origin=orig, width=2.10, length=1, placement="lowerleft")
d7 = Topology.Rotate(d7, origin=orig, axis=[0,1,0], angle=-90)

orig = Vertex.ByCoordinates(3.3,2,0.05)
d8 = Face.Rectangle(origin=orig, width=1.4, length=2.10, placement="lowerleft")
d8 = Topology.Rotate(d8, origin=orig, axis=[1,0,0], angle=90)
d9 = Topology.Translate(d8, 0, 9.2, 0)
door_centroid = Topology.Centroid(d8)
gd_flr_drs = [d2,d3,d4,d5,d6,d7]
fst_flr_dr = [Topology.Translate(door,0,0,3) for door in gd_flr_drs]
drs = gd_flr_drs + fst_flr_dr + [d8,d9]

# Set the dictionaries for the doors
doors = []
for door in drs:
    uuid = Topology.UUID(door, namespace=namespace)
    d = Dictionary.ByKeysValues(["label", "name", "type"], [str(uuid), "Door", "Door"])
    door = Topology.SetDictionary(door, d)
    doors.append(door)
door_cluster = Cluster.ByTopologies(doors)

# Build the windows
decomp = CellComplex.Decompose(house)
outer_walls = decomp["externalVerticalFaces"]
windows = []
for outer_wall in outer_walls:
    window = Topology.Scale(outer_wall, origin=Topology.Centroid(outer_wall), x=0.3, y=0.3, z=0.3)
    window = Topology.Translate(window, x=0, y=0, z=-0.4)
    uuid = Topology.UUID(door, namespace=namespace)
    d = Dictionary.ByKeysValues(["label", "name", "type"], [str(uuid), "Window", "Window"])
    window = Topology.SetDictionary(window, d)
    windows.append(window)
windows = [w for w in windows if Vertex.Distance(Topology.Centroid(w),door_centroid) > 0.2]
windows = [w for w in windows if Face.Area(w) > 0.5]

window_cluster = Cluster.ByTopologies(windows)
# Add the doors and windows to the house
house = Topology.AddApertures(house, doors+windows, exclusive=True, subTopologyType="face", tolerance=0.001)

# Add Furniture
bed = Cell.Prism(origin=Vertex.ByCoordinates(7.8,2,0), width=2, length=1.4, height=0.6, placement="bottom")
uuid = Topology.UUID(bed, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Bed"])
bed = Topology.SetDictionary(bed, d)

st1 = Cell.Prism(origin=Vertex.ByCoordinates(8.55,0.9,0), width=0.5, length=0.5, height=0.6, placement="bottom")
uuid = Topology.UUID(st1, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Side Table"])
st1 = Topology.SetDictionary(st1, d)

st2 = Cell.Prism(origin=Vertex.ByCoordinates(8.55,3.1,0), width=0.5, length=0.5, height=0.6, placement="bottom")
uuid = Topology.UUID(st2, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Side Table"])
st2 = Topology.SetDictionary(st2, d)

desk1 = Cell.Prism(origin=Vertex.ByCoordinates(8.4,7,0), width=0.7, length=1.5, height=0.6, placement="bottom")
uuid = Topology.UUID(desk1, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Desk"])
desk1 = Topology.SetDictionary(desk1, d)

ch1 = Cell.Prism(origin=Vertex.ByCoordinates(7.7,7,0), width=0.5, length=0.5, height=0.6, placement="bottom")
uuid = Topology.UUID(ch1, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Chair"])
ch1 = Topology.SetDictionary(ch1, d)

desk2 = Cell.Prism(origin=Vertex.ByCoordinates(6,8.4,0), width=1.5, length=0.7, height=0.6, placement="bottom")
uuid = Topology.UUID(desk2, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Desk"])
desk2 = Topology.SetDictionary(desk2, d)

ch2 = Cell.Prism(origin=Vertex.ByCoordinates(5.95,7.7,0), width=0.5, length=0.5, height=0.6, placement="bottom")
uuid = Topology.UUID(ch2, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Chair"])
ch2 = Topology.SetDictionary(ch2, d)

toilet = Cell.Prism(origin=Vertex.ByCoordinates(6.5,5.5,0), width=0.5, length=0.65, height=0.6, placement="bottom")
uuid = Topology.UUID(toilet, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Toilet"])
toilet = Topology.SetDictionary(toilet, d)

sink = Cell.Prism(origin=Vertex.ByCoordinates(5.6,5.6,0), width=0.6, length=0.5, height=0.6, placement="bottom")
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Sink"])
sink = Topology.SetDictionary(sink, d)

tub = Cell.Prism(origin=Vertex.ByCoordinates(7.45,5,0), width=0.8, length=1.7, height=0.6, placement="bottom")
uuid = Topology.UUID(tub, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Tub"])
tub = Topology.SetDictionary(tub, d)

sofa1 = Cell.Prism(origin=Vertex.ByCoordinates(1.5,2.8,0), width=0.8, length=0.8, height=0.6, placement="bottom")
uuid = Topology.UUID(sofa1, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Sofa"])
sofa1 = Topology.SetDictionary(sofa1, d)

sofa2 = Cell.Prism(origin=Vertex.ByCoordinates(0.6,1.25,0), width=0.8, length=2, height=0.6, placement="bottom")
uuid = Topology.UUID(sofa2, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Sofa"])
sofa2 = Topology.SetDictionary(sofa2, d)

st3 = Cell.Prism(origin=Vertex.ByCoordinates(0.7,2.65,0), width=0.5, length=0.5, height=0.6, placement="bottom")
uuid = Topology.UUID(st3, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Side Table"])
st3 = Topology.SetDictionary(st3, d)

tv = Cell.Prism(origin=Vertex.ByCoordinates(2.6,1.25,0), width=0.4, length=1, height=0.6, placement="bottom")
uuid = Topology.UUID(tv, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "TV"])
tv = Topology.SetDictionary(tv, d)

kt = Cell.Prism(origin=Vertex.ByCoordinates(1.5,6.5,0), width=1, length=1.4, height=0.6, placement="bottom")
uuid = Topology.UUID(kt, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Kitchen Island"])
kt = Topology.SetDictionary(kt, d)

cb = Cell.Prism(origin=Vertex.ByCoordinates(1.5,8.6,0), width=2.8, length=0.6, height=0.6, placement="bottom")
uuid = Topology.UUID(cb, namespace=namespace)
d = Dictionary.ByKeysValues(["label", "name"], [str(uuid), "Kitchen Cabinets"])
cb = Topology.SetDictionary(cb, d)

furniture_gd = [bed, st1, st2, desk1, ch1, desk2, ch2, toilet, sink, tub, sofa1, sofa2, st3, tv, kt, cb]
furniture_1 = [Topology.Translate(f, 0, 0, 3) for f in furniture_gd]
for i, f in enumerate(furniture_1):
    d = Topology.Dictionary(furniture_gd[i])
    uuid = Topology.UUID(f, namespace=namespace)
    d = Dictionary.SetValueAtKey(d, "label", str(uuid))
    f = Topology.SetDictionary(f, d)
furniture = furniture_gd + furniture_1
house = Topology.AddContent(house, furniture, subTopologyType="cell", tolerance=0.001)
house = Topology.TransferDictionariesBySelectors(house, selectors, tranCells=True)

d = {
    "cells": "Space",
    "externalVerticalFaces": "External_Vertical_Wall",
    "internalVerticalFaces": "Internal_Vertical_Wall",
    "topHorizontalFaces": "Roof_Slab",
    "bottomHorizontalFaces": "Ground_Slab",
    "internalHorizontalFaces": "Internal_Floor_Slab",
    "externalInclinedFaces": "External_Inclined_Wall",
    "internalInclinedFaces": "Internal_Inclined_Wall",
    "externalVerticalApertures": "External_Vertical_Window",
    "internalVerticalApertures": "Internal_Vertical_Door",
    "topHorizontalApertures": "Skylight_Roof_Window",
    "bottomHorizontalApertures": "Floor_Window",
    "internalHorizontalApertures": "Floor_Window",
    "externalInclinedApertures": "External_Inclined_Window",
    "internalInclinedApertures": "Internal_Inclined_Window",
}
#Assign the appropriate element types
_ = assignTypesAndLabels(house, d)
print("Done building the house.")

Key.Lower: cells
Key.Lower: cells
Key.Lower: cells
Key.Lower: cells
Key.Lower: cells
Key.Lower: cells
Key.Lower: cells
Key.Lower: cells
Key.Lower: cells
Key.Lower: cells
Key.Lower: cells
Key.Lower: cells
Key.Lower: cells
Key.Lower: externalverticalfaces
Name Value: External Wall
Key.Lower: externalverticalfaces
Name Value: External Wall
Key.Lower: externalverticalfaces
Name Value: External Wall
Key.Lower: externalverticalfaces
Name Value: External Wall
Key.Lower: externalverticalfaces
Name Value: External Wall
Key.Lower: externalverticalfaces
Name Value: External Wall
Key.Lower: externalverticalfaces
Name Value: External Wall
Key.Lower: externalverticalfaces
Name Value: External Wall
Key.Lower: externalverticalfaces
Name Value: External Wall
Key.Lower: externalverticalfaces
Name Value: External Wall
Key.Lower: externalverticalfaces
Name Value: External Wall
Key.Lower: externalverticalfaces
Name Value: External Wall
Key.Lower: externalverticalfaces
Name Value: External Wall
Key.Lower: e

## Visualize the Graph of the House

In [47]:
house_graph = Graph.ByTopology(house,
                     direct=False,
                     directApertures=True,
                     viaSharedTopologies=True,
                     viaSharedApertures=True,
                     toExteriorTopologies=True,
                     toExteriorApertures=True,
                     toContents = True,
                     storeBREP = False)
house_graph = Graph.Flatten(house_graph, k=0.15)
data01 = Plotly.DataByTopology(Cluster.ByTopologies(Topology.Edges(house)))
data02 = Plotly.DataByTopology(door_cluster, faceColor="brown", faceOpacity=0.1)
data03 = Plotly.DataByTopology(window_cluster, faceColor="lightblue", faceOpacity=0.3)
data04 = Plotly.DataByTopology(Cluster.ByTopologies(Topology.Edges(Cluster.ByTopologies(furniture))), faceColor="yellow", faceOpacity=0.1)
vertexGroups = ["Living Room", "Kitchen", "Bedroom", "Bathroom", "Corridor", "Home Office", "StairWell", "External Wall", "Internal Wall", "Roof Slab", "Ground Slab", "Internal Slab", "Door", "Window", "Desk", "Bed", "Side Table", "Tub", "Kitchen Island", "Kitchen Cabinets", "Sink", "Toilet", "Sofa", "Chair", "TV"]
data05= Plotly.DataByGraph(house_graph, vertexSize=8, vertexColor="red", edgeWidth=2, vertexLabelKey="name", vertexGroupKey="name", vertexGroups=vertexGroups, colorScale="viridis_r")

#figure = Plotly.FigureByData(data01+data02+data03+data04+data05, width=1024, height=960)
figure = Plotly.FigureByData(data05, width=1024, height=960)
Plotly.Show(figure)

Number of Nodes: 178


## Check the Dictionaries of the Cells

In [7]:
house_cells = Topology.Cells(house)
for house_cell in house_cells:
    d = Topology.Dictionary(house_cell)
    print(Dictionary.Keys(d), Dictionary.Values(d))

['hasVolume', 'label', 'name', 'type'] [45.0, 'ae768ccc-b469-5df3-a6b7-417072acb0d4', 'Living Room', 'Space']
['hasVolume', 'label', 'name', 'type'] [36.0, '27d50689-7e2a-58bf-984d-ca2205cce94b', 'Kitchen', 'Space']
['hasVolume', 'label', 'name', 'type'] [36.0, '1a4a5e1e-6010-59a8-85c1-33282b8663bb', 'Corridor', 'Space']
['hasVolume', 'label', 'name', 'type'] [45.0, 'e7192c5a-32aa-5f00-bdcd-ab5bf5c10728', 'Living Room', 'Space']
['hasVolume', 'label', 'name', 'type'] [36.0, '305fc232-fbee-5876-a24c-69d2f91ea8e7', 'Kitchen', 'Space']
['hasVolume', 'label', 'name', 'type'] [30.72, '4c195346-b850-58c3-b045-db0c566cb3b6', 'Stairwell', 'Space']
['hasVolume', 'label', 'name', 'type'] [36.0, 'e8ea9c39-bc3e-5bd0-bc94-b56a821c5573', 'Home Office', 'Space']
['hasVolume', 'label', 'name', 'type'] [18.0, '8e0e063d-3150-5f4b-8fe9-5aa2c7f8c9f8', 'Bathroom', 'Space']
['hasVolume', 'label', 'name', 'type'] [48.0, 'ae5fe3e9-3352-509c-84ea-523d885f098e', 'Bedroom', 'Space']
['hasVolume', 'label', 'name'

## Export the Graph to the NEO4J Graph Database

### Enter your credentials. Make sure you have your Neo4J DBMS running and that you know the URL, username, and password

In [8]:
print("Enter your credentials!")
default_url = "bolt://localhost:7687"
url = input(f"Enter the Neo4j DBMS URL (default: {default_url}): ") or default_url

default_username = "neo4j"
# Input username
# Prompt the user for the username, allowing for a default
username = input(f"Enter your Neo4j username (default: {default_username}): ") or default_username

# Input password (hidden)
password = getpass("Enter your Neo4j password: ")
print("Done. Move to next step")

Enter your credentials!
Done. Move to next step


### Get the Neo4J Graph (in reality this is the driver, but we can think of it as the neo4j graph)

In [9]:
n_graph = Neo4j.ByParameters(url, username, password)
print("I got the Graph Database:", n_graph)

I got the Graph Database: <neo4j._sync.driver.BoltDriver object at 0x00000280E6E2B750>


### Reset the graph (Delete all existing nodes and edges. Be careful with this step if you need to keep your graph)

In [10]:
n_graph = Neo4j.Reset(n_graph)
print("The Graph Database has been reset")

The Graph Database has been reset


### Convert the Topologic Graph to a Neo4J graph.

In [11]:
n_graph = Neo4j.ByGraph(n_graph,
                           house_graph,
                           vertexLabelKey="label",
                           defaultVertexLabel="Untitled",
                           vertexCategoryKey="type",
                           defaultVertexCategory=None,
                           edgeLabelKey="relationship",
                           defaultEdgeLabel="CONNECTED_TO",
                           edgeCategoryKey= "category",
                           defaultEdgeCategory=None,
                           mantissa=6,
                           tolerance = 0.0001)
print(n_graph)